In [1]:
import clipboard, os
from officelib.xllib import *
def crp():
    clipboard.copy(repr(clipboard.paste()))

In [18]:
crp()

In [3]:
fdir = 'D:\\Write Cache Test Runner\\Evo'
xl = Excel()
mwb = xl.Workbooks.Add()
mws = mwb.Worksheets(1)
row = 2
headers = [('Query Drive', 'Report Settings Drive', 'Output Drive', 'Batch ID', 'Total(s)', 'Batch Info(s)', 'Execute Query(s)', 'Parse Report(s)', 'Write Disk(s)', 'Sustained Write/sec', 'Write Cache', 'WPS Req', "WPS %")]
mws.Cells.Range(mws.Cells(1,1), mws.Cells(1, len(headers[0]))).Value = headers
mws.Name = "Data"
# Delete the last two worksheets
mwb.Worksheets(3).Delete()
mwb.Worksheets(2).Delete()

def put(wps, yn, v):
    global row
    c = mws.Cells(row, 1)
    nr = len(v)
    nc = len(v[0])
    c2 = c.Offset(nr, nc)
    mws.Cells.Range(c, c2).Value = v
    yns = ["no cache", 'cache'][yn == "with"]
    fill = [(yns, wps)]
    #import pdb;pdb.set_trace()
    mws.Cells.Range(c.Offset(1, nc+1), c2.Offset(1, 3)).Value = fill
#     for i in range(nr):
#         mws.Cells(i + row, nc+1).Value = yns
#         mws.Cells(i + row, nc+2).Value = wps
    row += nr
    
with screen_lock(xl):
    for fp in os.listdir(fdir):
        print("Analyzing '%s'... "%fp, end="")
        parts = fp.split()
        wps = parts[3]
        yn = parts[6]
        wb = xl.Workbooks.Open(os.path.join(fdir, fp))
        ws = wb.Worksheets(1)
        cells = ws.Cells
        c1 = cells(2, 1)
        c2 = c1.End(xlc.xlDown).End(xlc.xlToRight)
        v = cells.Range(c1, c2).Value
        put(wps, yn, v)
        wb.Close()
        print("Done")
    print("Inserting 'WPS %' column... ", end="")
    wpsc1 = mws.Cells(1,1).End(xlc.xlToRight).Offset(2,1)
    wpsc1.Value = "=%s/%s*100" % (wpsc1.Offset(1, -2).GetAddress(False,False), wpsc1.Offset(1, 0).GetAddress(False,False))
    c2 = wpsc1.Offset(1, 0).End(xlc.xlDown).Offset(1, 2)
    wpsc1.AutoFill((mws.Cells.Range(wpsc1, c2)))
    print("Done")

Analyzing 'write cache test 1 per sec no cache.csv'... Done
Analyzing 'write cache test 1 per sec with cache.csv'... Done
Analyzing 'write cache test 10 per sec no cache.csv'... Done
Analyzing 'write cache test 10 per sec with cache.csv'... Done
Analyzing 'write cache test 100 per sec no cache.csv'... Done
Analyzing 'write cache test 100 per sec with cache.csv'... Done
Analyzing 'write cache test 1000 per sec no cache.csv'... Done
Analyzing 'write cache test 1000 per sec with cache.csv'... Done
Inserting 'WPS %' column... Done


In [4]:
print("Preparing to insert pivot tables")

ur = "%s!%s"%(mws.Name, mws.UsedRange.GetAddress(ReferenceStyle=xlc.xlR1C1))
cache = mwb.PivotCaches().Create(SourceType=xlc.xlDatabase, SourceData=ur, Version=xlc.xlPivotTableVersion12)
mwb.ShowPivotChartActiveFields=True

total_cfg = [
    # WS Name,  yvar Name, xvars, xlabel, ylabel, coord
    ('ReportGenByCache', "Report Generation Speed", "Total(s)", ["Write Cache"], ["Write Cache"], "Report Generation Time(s)", (0.0, 0.0, 360.0, 211.0)),
    ('ReportGenByCacheBatchID', "Report Generation Speed", "Total(s)", ["Batch ID", "Write Cache"], ["Batch ID", "Write Cache"], "Report Generation Time(s)", (0.0, 215.39999389648438, 360.0, 211.0)),
    ('ReportGenByCacheWPSBatchID', "Report Generation Speed", "Total(s)", ["Batch ID", "WPS Req", "Write Cache"], ["Batch ID", "Writes / Sec Requested", "Write Cache"], "Report Generation Time(s)", (360.6000061035156, 0.0, 598.8, 423.4)),
]

wps_cfg = [
    # WS Name,  title, yvar Name, xvars, xlabel, ylabel, coord
    ('WPSByCache', "Write Speed", "Sustained Write/sec", ["Write Cache"], ["Write Cache"], "Writes per Second", (0.0, 0.0, 360.0, 211.0)),
    ('WPSByCacheBatchID', "Write Speed", "Sustained Write/sec", ["Batch ID", "Write Cache"], ["Batch ID", "Write Cache"], "Writes per Second", (0.0, 215.39999389648438, 360.0, 211.0)),
    ('WPSByCacheWPSBatchID', "Write Speed", "Sustained Write/sec", ["Batch ID", "WPS Req", "Write Cache"], ["Batch ID", "Writes / Sec Requested", "Write Cache"], "Writes per Second", (360.6000061035156, 0.0, 598.8, 423.4)),
]

def _mk_pvts(cfg, sheetname):
    print("Preparing to create pivot tables for '%s'"%sheetname)
    summary = mwb.Worksheets.Add()
    summary.Name = sheetname
    summary.Move(After=mwb.Worksheets(mwb.Worksheets.Count - 1))
    print("Preparing to create pivot tables for '%s'"%sheetname)
    for i, (wsn, title, yvar, xvars, xlabels, ylabel, coord) in enumerate(cfg, 1):
        print("Preparing table %d/%d"%(i, len(cfg)))
        
        sh = mwb.Worksheets.Add()
        sh.Name = wsn
        
        pt = cache.CreatePivotTable(TableDestination="%s!R1C1"%sh.Name, TableName="PT"+wsn, DefaultVersion=xlc.xlPivotTableVersion12)
        ch = sh.Shapes.AddChart().Chart
        ch.SetSourceData(Source=sh.Cells.Range("$A$1:$C$18"))
        ch.ChartType = xlc.xlColumnClustered
        
        co = ch.Parent
        co.Width, co.Height = coord[2:]
        co.Left, co.Top = 170, 12
        
        pt.AddDataField(pt.PivotFields(yvar), "Ave of %s"%yvar, xlc.xlAverage)
        for i, xv in enumerate(xvars, 1):
            xvf = pt.PivotFields(xv)
            xvf.Orientation = xlc.xlRowField
            xvf.Position = i
        
        xax = ch.Axes(xlc.xlCategory)
        xax.HasTitle = True
        xax.AxisTitle.Text = "\n".join(reversed(xlabels))
        yax = ch.Axes(xlc.xlValue)
        yax.HasTitle = True
        yax.AxisTitle.Text = ylabel
        ch.HasLegend=False
        ch.HasTitle = True
        ch.ChartTitle.Text = title
        
        print("Copying and resizing.")
        co.Copy()
        summary.Paste()
        ncho = summary.ChartObjects(i)
        ncho.Left, ncho.Top, ncho.Width, ncho.Height = coord
    
    print("Done")
    return summary
    
def mk_pvts(cfg, name):
    return _mk_pvts(cfg, name)
    
with screen_lock(xl):
    s1 = mk_pvts(total_cfg, "Summary - Report Generation")
    s2 = mk_pvts(wps_cfg, "Summary - Database Writes")

    s1.Move(After=mwb.Worksheets(mwb.Worksheets.Count - 1))
    s2.Move(After=s1)

Preparing to insert pivot tables
Preparing to create pivot tables for 'Summary - Report Generation'
Preparing to create pivot tables for 'Summary - Report Generation'
Preparing table 1/3
Copying and resizing.
Preparing table 2/3
Copying and resizing.
Preparing table 3/3
Copying and resizing.
Done
Preparing to create pivot tables for 'Summary - Database Writes'
Preparing to create pivot tables for 'Summary - Database Writes'
Preparing table 1/3
Copying and resizing.
Preparing table 2/3
Copying and resizing.
Preparing table 3/3
Copying and resizing.
Done


In [87]:
s = xl.Selection
s.Left, s.Top

(169.1999969482422, 12.600000381469727)

In [104]:
xl.Selection.Parent.HasTitle

True